In [1]:
from simple.ohlc import debounce, midPrice, TTrade
from simple.plotly import chartFigure
from simple.funcs import symlog
import numpy as np

In [2]:
T = np.load('data/tick.npz')['BTCUSDT'].view(np.recarray)[:13000]
T[-5:]

rec.array([('2021-06-01T01:10:04.112000', '2021-06-01T01:10:04.157000', 37678.5,  -1100., nan),
           ('2021-06-01T01:10:04.112000', '2021-06-01T01:10:04.157000', 37678.5,   -900., nan),
           ('2021-06-01T01:10:04.112000', '2021-06-01T01:10:04.157000', 37678.5, -14856., nan),
           ('2021-06-01T01:10:04.230000', '2021-06-01T01:10:04.347000', 37678.5, -16856., nan),
           ('2021-06-01T01:10:04.232000', '2021-06-01T01:10:04.347000', 37678.5,  -7288., nan)],
          dtype=[('DateTimeA', '<M8[us]'), ('LocalTimeA', '<M8[us]'), ('PriceA', '<f8'), ('VolumeA', '<f8'), ('OpenIntA', '<f8')])

In [3]:
Buy = T[T.VolumeA > 0]
Sell = T[T.VolumeA < 0]

In [4]:
C = debounce(T)
C[-5:]

rec.array([('2021-06-01T01:09:21.633000', 2579, 37657.75,  1674999, 74400., 73400.,  -1000.,  8,  7, 1),
           ('2021-06-01T01:09:23.502000', 2580, 37662.25,        0, 50000., 50000.,      0.,  1,  1, 0),
           ('2021-06-01T01:09:23.502000', 2581, 37670.75,        0,  5000.,  5000.,      0.,  1,  1, 0),
           ('2021-06-01T01:09:23.729000', 2582, 37671.75,   111000,  7300.,  7300.,      0.,  6,  6, 0),
           ('2021-06-01T01:09:23.945000', 2583, 37678.75, 40287000, 57620., 16394., -41226., 25, 18, 7)],
          dtype=[('DT', '<M8[us]'), ('Index', '<i4'), ('Price', '<f8'), ('Duration', '<m8[us]'), ('Volume', '<f8'), ('BuySize', '<f8'), ('SellSize', '<f8'), ('Count', '<i4'), ('BuyCount', '<i4'), ('SellCount', '<i4')])

In [5]:
len(T), len(C)

(13000, 2584)

In [6]:
# indexed chart
fig = chartFigure(height=400,
    Sell=dict(x=Sell.DateTimeA, y=Sell.PriceA, color='red', mode='markers', size=-symlog(Sell.VolumeA)+3, opacity=0.2),
    Buy=dict(x=Buy.DateTimeA, y=Buy.PriceA, color='green', mode='markers', size=symlog(Buy.VolumeA)+3, opacity=0.2),
    Debounce=dict(mode='markers+lines', x=C.DT, y=C.Price, color='black', connectgaps=True, opacity=0.7)
)
fig

FigureWidgetResampler({
    'data': [{'marker': {'color': 'red',
                         'opacity': 0.2,
    …

In [7]:
C.Volume = C.BuySize + C.SellSize
Buy = C[C.Volume > 0]
Sell = C[C.Volume < 0]
Sell[-5:]

rec.array([('2021-06-01T01:08:10.270000', 2561, 37646.75,        0,   -212.,     0.,   -212.,  1,  0,  1),
           ('2021-06-01T01:08:59.130000', 2574, 37671.75,   103000,  -8200.,     0.,  -8200.,  6,  0,  6),
           ('2021-06-01T01:09:00.441000', 2575, 37657.75,  9318000, -22497.,     0., -22497., 11,  0, 11),
           ('2021-06-01T01:09:09.792000', 2576, 37650.25,        0,    -60.,     0.,    -60.,  2,  0,  2),
           ('2021-06-01T01:09:23.945000', 2583, 37678.75, 40287000, -24832., 16394., -41226., 25, 18,  7)],
          dtype=[('DT', '<M8[us]'), ('Index', '<i4'), ('Price', '<f8'), ('Duration', '<m8[us]'), ('Volume', '<f8'), ('BuySize', '<f8'), ('SellSize', '<f8'), ('Count', '<i4'), ('BuyCount', '<i4'), ('SellCount', '<i4')])

In [8]:
# indexed chart without bidask
fig = chartFigure(height=400,
    Sell=dict(x=Sell.Index, y=Sell.Price, color='red', mode='markers', size=-symlog(Sell.Volume)-1, opacity=0.4),
    Buy=dict(x=Buy.Index, y=Buy.Price, color='green', mode='markers', size=symlog(Buy.Volume)-1, opacity=0.4),
)
fig

FigureWidgetResampler({
    'data': [{'marker': {'color': 'red',
                         'opacity': 0.4,
    …

In [9]:
T = np.zeros(len(C), dtype=TTrade).view(np.recarray)
T.DateTimeA = C.DT
T.PriceA = C.Price
T.VolumeA = C.Volume
T[-5:]

rec.array([('2021-06-01T01:09:21.633000', '1970-01-01T00:00:00.000000', 37657.75,  72400., 0.),
           ('2021-06-01T01:09:23.502000', '1970-01-01T00:00:00.000000', 37662.25,  50000., 0.),
           ('2021-06-01T01:09:23.502000', '1970-01-01T00:00:00.000000', 37670.75,   5000., 0.),
           ('2021-06-01T01:09:23.729000', '1970-01-01T00:00:00.000000', 37671.75,   7300., 0.),
           ('2021-06-01T01:09:23.945000', '1970-01-01T00:00:00.000000', 37678.75, -24832., 0.)],
          dtype=[('DateTimeA', '<M8[us]'), ('LocalTimeA', '<M8[us]'), ('PriceA', '<f8'), ('VolumeA', '<f8'), ('OpenIntA', '<f8')])

In [10]:
C = debounce(T)
len(T), len(C)

(2584, 2565)

In [11]:
# indexed chart without bidask
fig = chartFigure(height=400,
    Sell=dict(x=Sell.Index, y=Sell.Price, color='red', mode='markers', size=-symlog(Sell.Volume)-1, opacity=0.4),
    Buy=dict(x=Buy.Index, y=Buy.Price, color='green', mode='markers', size=symlog(Buy.Volume)-1, opacity=0.4),
)
fig

FigureWidgetResampler({
    'data': [{'marker': {'color': 'red',
                         'opacity': 0.4,
    …